In [ ]:
!pip install keras_preprocessing

In [ ]:
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os

In [ ]:
# Unduh dan ekstrak dataset
data_url = 'https://drive.usercontent.google.com/download?id=1T2vteOAvBrj23MK3ry_ynLWli4_gNbqe&export=download&confirm=t'
urllib.request.urlretrieve(data_url, 'Datasets.zip')
with zipfile.ZipFile('Datasets.zip', 'r') as zip_ref:
    zip_ref.extractall('Datasets')

base_dir = 'Datasets'
train_path = os.path.join(base_dir, 'Dataset/train')
val_path = os.path.join(base_dir, 'Dataset/validation')
test_path = os.path.join(base_dir, 'Dataset/test')

In [ ]:
# ImageDataGenerator untuk training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)

# ImageDataGenerator untuk validasi & testing
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

validation_generator = val_test_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical'
)

In [ ]:
# Load model Xception dan lakukan fine-tuning 30 layer terakhir
pre_trained_model = Xception(input_shape=(299, 299, 3), include_top=False, weights='imagenet')
for layer in pre_trained_model.layers[:-30]:
    layer.trainable = False
for layer in pre_trained_model.layers[-30:]:
    layer.trainable = True

last_layer_output = pre_trained_model.output
print(f'Output shape of the Xception base model: {last_layer_output.shape}')

In [ ]:
# Bangun model klasifikasi
x = layers.GlobalAveragePooling2D()(last_layer_output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.1)(x)
output = layers.Dense(3, activation='softmax')(x)

model = Model(inputs=pre_trained_model.input, outputs=output)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['acc']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('Xception_model.h5', monitor='val_acc', save_best_only=True, verbose=1)

# Training
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.4098 - loss: 1.0696
Epoch 1: val_acc improved from -inf to 0.65101, saving model to Xception_model.h5


17/17 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - acc: 0.4137 - loss: 1.0675 - val_acc: 0.6510 - val_loss: 0.9689 - learning_rate: 1.0000e-04
Epoch 2/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.6591 - loss: 0.8575
Epoch 2: val_acc improved from 0.65101 to 0.75503, saving model to Xception_model.h5


17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.6605 - loss: 0.8549 - val_acc: 0.7550 - val_loss: 0.7652 - learning_rate: 1.0000e-04
Epoch 3/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.7269 - loss: 0.6843
Epoch 3: val_acc improved from 0.75503 to 0.78859, saving model to Xception_model.h5


17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.7282 - loss: 0.6820 - val_acc: 0.7886 - val_loss: 0.5970 - learning_rate: 1.0000e-04
Epoch 4/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.7968 - loss: 0.5306
Epoch 4: val_acc improved from 0.78859 to 0.81544, saving model to Xception_model.h5


...

17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.8096 - loss: 0.4756 - val_acc: 0.8322 - val_loss: 0.4420 - learning_rate: 1.0000e-04
Epoch 6/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.8378 - loss: 0.4206
Epoch 6: val_acc improved from 0.83221 to 0.85570, saving model to Xception_model.h5


17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.8374 - loss: 0.4205 - val_acc: 0.8557 - val_loss: 0.3617 - learning_rate: 1.0000e-04
Epoch 7/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.8572 - loss: 0.3476
Epoch 7: val_acc improved from 0.85570 to 0.86577, saving model to Xception_model.h5


17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.8569 - loss: 0.3483 - val_acc: 0.8658 - val_loss: 0.3591 - learning_rate: 1.0000e-04
Epoch 8/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.8706 - loss: 0.3374
Epoch 8: val_acc did not improve from 0.86577
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.8707 - loss: 0.3368 - val_acc: 0.8591 - val_loss: 0.3386 - learning_rate: 1.0000e-04
Epoch 9/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.8941 - loss: 0.3139
Epoch 9: val_acc did not improve from 0.86577
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.8943 - loss: 0.3132 - val_acc: 0.8523 - val_loss: 0.3664 - learning_rate: 1.0000e-04
Epoch 10/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.8874 - loss: 0.2724
Epoch 10: val_acc did not improve from 0.86577
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.8875 - loss: 0.2724 - val_acc: 0.8557 - val_loss: 0.3115 - learning_rate: 1.0000e-04
Epoch 11/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.9181 - loss: 0.2180
Epoch 11: v

17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - acc: 0.9354 - loss: 0.1887 - val_acc: 0.8725 - val_loss: 0.4446 - learning_rate: 1.0000e-04
Epoch 15/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.9342 - loss: 0.1557
Epoch 15: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.

Epoch 15: val_acc did not improve from 0.87248
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9340 - loss: 0.1559 - val_acc: 0.8624 - val_loss: 0.4311 - learning_rate: 1.0000e-04
Epoch 16/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.9401 - loss: 0.1636
Epoch 16: val_acc did not improve from 0.87248
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - acc: 0.9402 - loss: 0.1632 - val_acc: 0.8557 - val_loss: 0.4005 - learning_rate: 2.0000e-05
Epoch 17/120
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - acc: 0.9568 - loss: 0.1445
Epoch 17: val_acc did not improve from 0.87248
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.9570 - loss: 0.1437 - val_acc: 0.8557 - val_loss: 0.3677 - learning_rate: 2.0000e-05
Epoch 18

17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - acc: 0.9612 - loss: 0.1262 - val_acc: 0.8893 - val_loss: 0.3124 - learning_rate: 2.0000e-05


In [ ]:
train_path = os.path.join(base_dir, "Dataset/train")
val_path = os.path.join(base_dir, "Dataset/validation")
test_path = os.path.join(base_dir, "Dataset/test")

train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=128,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
validation_generator = validation_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=128,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=128,
    class_mode='categorical'
)

model = tf.keras.models.load_model('Xception_model.h5')

train_loss, train_acc = model.evaluate(train_generator, verbose=1)
val_loss, val_acc = model.evaluate(validation_generator, verbose=1)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print(f"Train Accuracy: {train_acc:.4f}, Loss: {train_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}, Loss: {val_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Ambil riwayat dari training
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Plot Akurasi
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Accuracy - Xception Model')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Loss - Xception Model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

In [ ]:
def load_and_process_image(image_path, target_size=(299, 299)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return img_array

def load_test_data(test_dir, class_names):
    data = []
    labels = []
    class_mapping = {name: idx for idx, name in enumerate(class_names)}
    for class_name in class_names:
        class_path = os.path.join(test_dir, class_name)
        if not os.path.isdir(class_path):
            print(f"Warning: Folder {class_path} not found.")
            continue
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            try:
                data.append(load_and_process_image(image_path))
                labels.append(class_mapping[class_name])
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")
    return np.array(data), np.array(labels)

In [ ]:
test_data_dir = val_path
class_names = sorted(os.listdir(test_data_dir))
X_test, y_test = load_test_data(test_data_dir, class_names)

model = keras.models.load_model('Xception_model.h5')

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(to_categorical(y_test, num_classes=len(class_names)), axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
class_report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Xception Model (Validation)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

print("\nClassification Report (Validation Data):")
print(class_report)

In [ ]:
test_data_dir = train_path
class_names = sorted(os.listdir(test_data_dir))
X_test, y_test = load_test_data(test_data_dir, class_names)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(to_categorical(y_test, num_classes=len(class_names)), axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
class_report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Xception Model (Training)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

print("\nClassification Report (Training Data):")
print(class_report)

In [ ]:
test_data_dir = test_path
class_names = sorted(os.listdir(test_data_dir))
X_test, y_test = load_test_data(test_data_dir, class_names)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(to_categorical(y_test, num_classes=len(class_names)), axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
class_report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Xception Model (Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

print("\nClassification Report (Test Data):")
print(class_report)